In [1]:
import cvxpy as cp
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection  import train_test_split
import matplotlib.pyplot as plt

In [2]:
X,y =make_blobs(n_samples=130, centers=2,n_features=3)
m,n=np.shape(X)
for i in range(len(y)):
    if (y[i]==0):
        y[i]=-1
X_train=X[:100]
y_train=y[:100]
X_test=X[100:]
y_test=y[100:]

In [423]:
import numpy as np
from sklearn.metrics.pairwise import rbf_kernel
class SVMCust:
    def __init__(self) -> None:
        theta=1
        self.W=0
        self.b=0
        self.a=0
        self.C=0
        self.m=0
    
    def kg(self,a, b, sigma=1):
        return np.exp((-(np.linalg.norm(a-b)**2))/(2*sigma**2))
        # # print("dot:",np.dot(a,b))
        # # val=np.linalg.det(np.dot(a,b))
        
        # # print("as",a.shape, "b:",b.shape)
        # # print("a",a,"b",b,"v:",val)
        # return np.dot(a,b)
    # generating kernal matrix
    def k_mat(self,x, k_func=kg):
        m = x.shape[0]
        mat = np.zeros((m, m))
        for i in range(m):
            for j in range(m):
                mat[i, j] = self.kg(x[i, :], x[j, :])
        return mat

    def getAlphas(self):
        return self.a
    def getWeight(self):
        return self.W
    def getBias(self):
        return self.b
    def train(self,X_train,y_train):
        print("shape:",X_train.shape)
        (m,f)=X_train.shape
        y_train=np.asarray(y_train)
        self.m=m
        X=np.array(X_train)
        # k = rbf_kernel(X)
        k=self.k_mat(X)
        print(np.all(np.abs(k-k.T) < 1e-08))
        # print(k)
        G=np.zeros((m,m))
        print("shaope:",G.shape)
        print("m:",m)
        for i in tqdm(range(0,m)):
            for j in range(0,m):
                # print("i:",i,"j:",j)
                # print("s:",(y[i]*(np.inner(X[i],np.transpose(X[j])))*y[j]).shape)
                G[i][j]=(y_train[i]*y_train[j]*(k[i][j]))
        # G=y*k*np.transpose(y)
       

        self.a = cp.Variable(m)
        self.C = cp.Parameter(nonneg=True)
        self.C.value = 0.02
        print("min eigne val:",np.linalg.eigvals(G).min())
        # G = cp.atoms.affine.wraps.psd_wrap(G)
        obj = cp.Minimize( (0.5) * cp.quad_form(self.a,G)-np.ones((1,m))@ self.a)
        constraints = [0<=self.a,self.a<=self.C ,np.transpose(y_train)@self.a==0]
        prob = cp.Problem(obj, constraints)
        print("Is prob dcp:",prob.is_dcp())
        result = prob.solve()
        print(self.a.value)
        self.W=np.zeros(f,dtype='float64')
        tempa=np.array(self.a.value)

        print(tempa)
        for i in range(m):
            if tempa[i]==self.C.value:
                self.W+=y_train[i]* tempa[i]* X[i]
        cnt_pos_vec=0
        # print("W:",type(W))
        self.W=np.asarray(self.W,dtype='float64')
        for i in range(m):
            if tempa[i]==self.C.value:
                # print(X[i].dtype)
                
                # print("y:",self.W.dtype)
                # print("h",y.dtype)
                # print((self.W* np.transpose(X[i])).shape)
                self.b+=y_train[i]-np.matmul(self.W, np.transpose(X[i]))
                cnt_pos_vec+=1
        self.b/=max(1,cnt_pos_vec)
    def predict(self,X):
        # ycap=np.matmul(self.W,np.transpose(X))
        matb=np.full((X.shape[0],1),self.b)
        matw=np.full((X.shape[0],X.shape[1]),self.W)
        # print(matb)
        # print(matw)
        # print("shapE,",np.matmul(X,np.transpose(self.W)).shape)
        print(np.matmul(X,np.transpose(self.W)))
        # print("fnal:,",np.add(matb,np.matmul(X,np.transpose(self.W))).shape)
        y_cap=self.b+np.transpose(np.matmul(X,np.transpose(self.W)))
        print(y_cap.shape)
        y_cap[y_cap<=0]=-1
        y_cap[y_cap>0]=1
        return y_cap
        
        
        

        
        
        



    
    
    
    

In [424]:
X_train

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
271,2,108,62,32,56,25.2,0.128,21
484,0,145,0,0,0,44.2,0.630,31
197,3,107,62,13,48,22.9,0.678,23
506,0,180,90,26,90,36.5,0.314,35
212,7,179,95,31,0,34.2,0.164,60
...,...,...,...,...,...,...,...,...
370,3,173,82,48,465,38.4,2.137,25
177,0,129,110,46,130,67.1,0.319,26
504,3,96,78,39,0,37.3,0.238,40
418,1,83,68,0,0,18.2,0.624,27


In [425]:
# scvmc=SVMCust()
# scvmc.train(X_train,y_train)
# print(scvmc.getAlphas().value)
# y_test
# scvmc.getWeight()
# scvmc.getBias()
# y_cap=scvmc.predict(X_test)
# y_test
# from sklearn.metrics import accuracy_score
# accuracy_score(y_test,y_cap)
import pandas as pd
data=pd.read_csv("datasets/diabetes.csv")
traindt=data;
test=data['Outcome']
traindt.drop('Outcome',axis=1,inplace=True)
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(traindt,test, test_size=0.25) # 70% training and 30% test
clf=SVMCust()
y_train[y_train==0]=-1
y_test[y_test==0]=-1
clf.train(X_train, y_train)
y_cap=clf.predict(X_test)

shape: (576, 8)
True
shaope: (576, 576)
m: 576


100%|██████████| 576/576 [00:00<00:00, 620.28it/s]


min eigne val: (0.9999311823288843+0j)
Is prob dcp: True
[0.01130435 0.01130435 0.01130435 0.01130435 0.02       0.01130435
 0.01130422 0.02       0.01130435 0.01130435 0.02       0.02
 0.01130435 0.01130435 0.02       0.01130435 0.01130435 0.02
 0.01130435 0.02       0.02       0.02       0.01130435 0.01130435
 0.02       0.01130435 0.01130435 0.01130435 0.01130435 0.01130435
 0.01130435 0.01130435 0.02       0.02       0.01130435 0.02
 0.01130435 0.01130435 0.01130435 0.01130435 0.01130435 0.02
 0.01130435 0.01130422 0.02       0.01130435 0.01130435 0.02
 0.01130435 0.02       0.02       0.02       0.02       0.02
 0.02       0.02       0.01130435 0.01130435 0.01130435 0.02
 0.01130435 0.01130435 0.02       0.01130435 0.01130435 0.01130435
 0.01130435 0.01130435 0.02       0.02       0.02       0.01130435
 0.02       0.02       0.02       0.02       0.02       0.01130435
 0.01130435 0.01130435 0.02       0.01130435 0.01130435 0.01130435
 0.01130435 0.02       0.01130435 0.01130435 0.

In [426]:
accuracy_score(y_test,y_cap)

0.6875

In [ ]:
y_cap

92    -1.0
368   -1.0
682   -1.0
153    1.0
442   -1.0
      ... 
493    1.0
121   -1.0
527   -1.0
402   -1.0
675   -1.0
Length: 192, dtype: float64

: 

In [20]:
from logging.config import valid_ident
from cvxpy import Problem
from tqdm import tqdm
class FairSVM:
    def __init__(self):
        self.W=[]
        self.b=[]
        self.a=[]
        self.C=[]
        self.m=0
    def kg(self,a, b, sigma=1):
        return np.exp((-(np.linalg.norm(a-b)**2))/(2*sigma**2))

    # generating kernal matrix
    def k_mat(self,x, k_func=kg):
        m = x.shape[0]
        mat = np.zeros((m, m))
        for i in range(m):
            for j in range(m):
                mat[i, j] = self.kg(x[i, :], x[j, :])
        return mat

    def getAlphas(self):
        return self.a
    def getWeight(self):
        return self.W
    def getBias(self):
        return self.b
    def fit(self,X_train,Y_train,list_of_sensetive_attr):
        (m,n)=X_train.shape
        y_train[y_train==0]=-1
        print("m:",m)
        Z=X_train.copy()
        non_sensetive_fet=[x for x in list(Z.columns) if x not in list_of_sensetive_attr]
        Z.drop( non_sensetive_fet,axis=1,inplace=True)
        X_train.drop( list_of_sensetive_attr, axis=1, inplace=True)
        print("dropped fet:")
        Z=np.asarray(Z)
        
        Galpha=np.zeros(m)
        Halpha=np.zeros(m)
        self.m=m

        X=np.array(X_train)
        # X = np.hstack((X, np.ones([m,1])))   
        # k = rbf_kernel(X)
        k=self.k_mat(X)
        # print(k)
        self.a = cp.Variable(m)
        self.C = cp.Parameter(nonneg=True)
        self.C.value = 0.02
        deltamat=np.zeros((m,m))
        print(self.C.value)
        # print(deltamat.value)
        print("h1:")
        for i in range (m):
            for j in range(m):
                if i==j:
                    deltamat[i][j]=(1/self.C.value)**1
                else:
                     deltamat[i][j]=1
        # objective=cp.Minimize(cp.sum(a)+ cp.sum(cp.multiply(a,y))
        res=0
        cres=0
        print("starting main loop:")
        for i in range(m):
            res+= self.a[i]+ \
            self.a[i]*y[i]*( cp.sum(cp.multiply(cp.multiply(self.a,y),k[i]))+ cp.sum(cp.multiply(cp.multiply(self.a,y),deltamat[i])))
            cres=(Z[i]-np.mean(Z))*cp.sum(cp.multiply(cp.multiply(self.a,y),k[i]))
        print("res:",res)
        objective=cp.Minimize(res)
        constraints=[cp.sum(cp.multiply(self.a,y))==0 , cres>= (-1)*self.C*m, cres <=self.C*m ]
        prob=cp.Problem(objective,constraints)
        print("is dcp:",prob.is_dcp())
        print("status:",prob.status)
        prob.solve()
            # for j in range(m):


        
        

        
        


In [22]:
X,y =make_blobs(n_samples=10, centers=2,n_features=2)
m,n=np.shape(X)
# for i in range(len(y)):
#     if (y[i]==0):
#         y[i]=-1
X_train=X
y_train=y
X_test=X[1500:]
y_test=y[1500:]


In [23]:
# from pyparsing import col
import pandas as pd

data=pd.DataFrame(data=X_train,columns=['a','b'])
data['one']=1

In [24]:
list(data.columns)

['a', 'b', 'one']

In [25]:
list_of_sensetive_fet=['b']

In [26]:
data.head()

,a,b,one
0,-2.490464,4.271891,1
1,-0.600075,2.711376,1
2,-5.486637,1.020512,1
3,-6.784746,-0.915751,1
4,-6.631578,-1.494234,1


In [27]:
y_train

array([1, 1, 0, 0, 0, 1, 1, 0, 1, 0])

In [28]:
fsvm=FairSVM()


In [29]:
fsvm.fit(data,y_test,list_of_sensetive_fet)

m: 10
dropped fet:
0.02
h1:
starting main loop:
res: var195[0] + var195[0] @ 1.0 @ (Sum(var195 @ [ 1.  1. -1. -1. -1.  1.  1. -1.  1. -1.] @ [1.00000000e+00 1.67497907e-01 1.12371961e-02 9.89962935e-05
 1.88877564e-04 9.99395685e-01 9.18611783e-01 2.29692875e-13
 6.94961030e-02 1.45810008e-07], None, False) + Sum(var195 @ [ 1.  1. -1. -1. -1.  1.  1. -1.  1. -1.] @ [50.  1.  1.  1.  1.  1.  1.  1.  1.  1.], None, False)) + var195[1] + var195[1] @ 1.0 @ (Sum(var195 @ [ 1.  1. -1. -1. -1.  1.  1. -1.  1. -1.] @ [1.67497907e-01 1.00000000e+00 6.52910158e-06 4.94408604e-09
 1.26007439e-08 1.78769333e-01 3.35294600e-01 2.09763268e-20
 9.15987354e-01 6.04436138e-13], None, False) + Sum(var195 @ [ 1.  1. -1. -1. -1.  1.  1. -1.  1. -1.] @ [ 1. 50.  1.  1.  1.  1.  1.  1.  1.  1.], None, False)) + var195[2] + var195[2] @ -1.0 @ (Sum(var195 @ [ 1.  1. -1. -1. -1.  1.  1. -1.  1. -1.] @ [1.12371961e-02 6.52910158e-06 1.00000000e+00 4.30613898e-01
 5.19211057e-01 1.01193161e-02 3.00347918e-03 2.1

DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
var195[0] @ 1.0 @ (Sum(var195 @ [ 1.  1. -1. -1. -1.  1.  1. -1.  1. -1.] @ [1.00000000e+00 1.67497907e-01 1.12371961e-02 9.89962935e-05
 1.88877564e-04 9.99395685e-01 9.18611783e-01 2.29692875e-13
 6.94961030e-02 1.45810008e-07], None, False) + Sum(var195 @ [ 1.  1. -1. -1. -1.  1.  1. -1.  1. -1.] @ [50.  1.  1.  1.  1.  1.  1.  1.  1.  1.], None, False))
var195[1] @ 1.0 @ (Sum(var195 @ [ 1.  1. -1. -1. -1.  1.  1. -1.  1. -1.] @ [1.67497907e-01 1.00000000e+00 6.52910158e-06 4.94408604e-09
 1.26007439e-08 1.78769333e-01 3.35294600e-01 2.09763268e-20
 9.15987354e-01 6.04436138e-13], None, False) + Sum(var195 @ [ 1.  1. -1. -1. -1.  1.  1. -1.  1. -1.] @ [ 1. 50.  1.  1.  1.  1.  1.  1.  1.  1.], None, False))
var195[2] @ -1.0 @ (Sum(var195 @ [ 1.  1. -1. -1. -1.  1.  1. -1.  1. -1.] @ [1.12371961e-02 6.52910158e-06 1.00000000e+00 4.30613898e-01
 5.19211057e-01 1.0